In [3]:
import numpy as np
from itertools import product

N = 5

In [16]:
num_range = np.arange(N)
coordinates = np.array(list(product(num_range, repeat=3)))

data = np.full((N**3, 4), -1)
data[:, :-1] = coordinates


In [18]:
data

array([[ 0,  0,  0, -1],
       [ 0,  0,  1, -1],
       [ 0,  0,  2, -1],
       [ 0,  0,  3, -1],
       [ 0,  0,  4, -1],
       [ 0,  1,  0, -1],
       [ 0,  1,  1, -1],
       [ 0,  1,  2, -1],
       [ 0,  1,  3, -1],
       [ 0,  1,  4, -1],
       [ 0,  2,  0, -1],
       [ 0,  2,  1, -1],
       [ 0,  2,  2, -1],
       [ 0,  2,  3, -1],
       [ 0,  2,  4, -1],
       [ 0,  3,  0, -1],
       [ 0,  3,  1, -1],
       [ 0,  3,  2, -1],
       [ 0,  3,  3, -1],
       [ 0,  3,  4, -1],
       [ 0,  4,  0, -1],
       [ 0,  4,  1, -1],
       [ 0,  4,  2, -1],
       [ 0,  4,  3, -1],
       [ 0,  4,  4, -1],
       [ 1,  0,  0, -1],
       [ 1,  0,  1, -1],
       [ 1,  0,  2, -1],
       [ 1,  0,  3, -1],
       [ 1,  0,  4, -1],
       [ 1,  1,  0, -1],
       [ 1,  1,  1, -1],
       [ 1,  1,  2, -1],
       [ 1,  1,  3, -1],
       [ 1,  1,  4, -1],
       [ 1,  2,  0, -1],
       [ 1,  2,  1, -1],
       [ 1,  2,  2, -1],
       [ 1,  2,  3, -1],
       [ 1,  2,  4, -1],


In [7]:
def set_data(coords: list, info: int):
  x, y, z = coords
  idx = x*(N**2) + y*N + z
  data[idx][-1] = info

In [8]:
def set_rel_data(coords: list, info: int):
  center = N // 2
  x, y, z = (np.array(coords) + center)

  idx = x*(N**2) + y*N + z
  data[idx][-1] = info

In [9]:
import plotly.express as px
fig = px.scatter_3d(
    data,
    x=data[:, 0],
    y=data[:, 1],
    z=data[:, 2],
    color=data[:, 3]
)
fig.show()

In [20]:
data_to_write = data[data[:, 3] != -1]
fig = px.scatter_3d(
    data_to_write,
    x=data_to_write[:, 0],
    y=data_to_write[:, 1],
    z=data_to_write[:, 2],
    color=data_to_write[:, 3]
)
fig.show()

In [11]:
def create_map(max_dist: int):
    N = max_dist*2 + 1
    map = np.full((N**3, 4), 1)
    num_range = np.arange(N)

    coordinates = np.array(list(product(num_range, repeat=3)))

    map[:, :-1] = coordinates

    return map

In [15]:
MAX_DIST = 20

map_data = create_map(MAX_DIST)
print(map_data)

[[ 0  0  0  1]
 [ 0  0  1  1]
 [ 0  0  2  1]
 ...
 [40 40 38  1]
 [40 40 39  1]
 [40 40 40  1]]


In [21]:
def get_hex_idxs(coords_map):
  return np.array(np.where(coords_map[:, 0] + coords_map[:, 1] + coords_map[:, 2] == MAX_DIST*3)).reshape(-1)


In [22]:
hexagon_idxs = get_hex_idxs(map_data)

hexagon_idxs

array([  860,   900,   940, ..., 67980, 68020, 68060])

In [25]:
hexagon_data = map_data[hexagon_idxs]
for hexagon in hexagon_data:

  dist = max(list(map(abs, [hexagon[i] - MAX_DIST for i in range(3)])))
  hexagon[-1] = dist



In [26]:
fig = px.scatter_3d(
    hexagon_data,
    x=hexagon_data[:, 0],
    y=hexagon_data[:, 1],
    z=hexagon_data[:, 2],
    color=hexagon_data[:, 3]
)
fig.show()

In [27]:
new_map = create_map(MAX_DIST)
hex_idxs = get_hex_idxs(new_map)
hex_data = new_map[hex_idxs]

In [28]:
fig = px.scatter_3d(
    hex_data,
    x=hex_data[:, 0],
    y=hex_data[:, 1],
    z=hex_data[:, 2],
    color=hex_data[:, 3]
)
fig.show()

In [32]:
ALL_MOVEMENTS = np.array([
    [0, 1, -1],
    [1, 0, -1],
    [1, -1, 0],
    [0, -1, 1],
    [-1, 0, 1],
    [-1, 1, 0]
])

In [33]:
import random

river_movements = ALL_MOVEMENTS[0:2]

RIVERS_COUNT = 40

MAX_RIVER_LENGTH = 7

RIVER_TYPE = 10

np.random.seed(42)
random.seed(42)

river_map = np.copy(new_map)

In [38]:
def is_within_bounds(point, max_value):
  return np.all(point >= 0) and np.all(point < max_value)


for _ in range(RIVERS_COUNT):
  river_length = random.randint(1, MAX_RIVER_LENGTH)
  start_point_idx = random.choice(hex_idxs)
  start_point = new_map[start_point_idx, :3]

  for _ in range(river_length):
    move = random.choice(river_movements)
    new_point = start_point + move

    if is_within_bounds(new_point, 2*MAX_DIST+1):
      new_point_idx = np.where((new_map[:, :3] == new_point).all(axis=1))[0]

      if new_point_idx.size > 0:
        new_point_idx = new_point_idx[0]
        river_map[new_point_idx, 3] = RIVER_TYPE
        start_point = new_point

In [39]:
hex_data = river_map[hex_idxs]

fig = px.scatter_3d(
    hex_data,
    x=hex_data[:, 0],
    y=hex_data[:, 1],
    z=hex_data[:, 2],
    color=hex_data[:, 3]
)
fig.show()

In [40]:
hills_map = np.copy(river_map)

HILLS_COUNT = 10

HILLS_TYPE = 100

In [41]:
for _ in range(HILLS_COUNT):
  start_point_idx = random.choice(hex_idxs)
  start_point = new_map[start_point_idx, :3]
  hills_map[start_point_idx, 3] = HILLS_TYPE

  for move in ALL_MOVEMENTS:
    new_point = start_point + move
    if is_within_bounds(new_point, 2*MAX_DIST+1):
      new_point_idx = np.where((new_map[:, :3] == new_point).all(axis=1))[0]
      hills_map[new_point_idx, 3] = HILLS_TYPE

In [42]:
PLAIN_TYPE = 1

types_map = {
    PLAIN_TYPE: "Plain",
    HILLS_TYPE: "Hill",
    RIVER_TYPE: "River",
}

vfunc = np.vectorize(types_map.get)

In [43]:
hex_data = hills_map[hex_idxs]

colors = vfunc(hex_data[:, 3])

fig = px.scatter_3d(
    hex_data,
    x=hex_data[:, 0],
    y=hex_data[:, 1],
    z=hex_data[:, 2],
    hover_name=hex_idxs,
    color=colors,
    color_discrete_sequence=["#387C44", "#3EA99F", "gray"],
)
fig.show()